In [1]:
#Predicting E-Commerce Product Recommendation Ratings from Reviews

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report

In [21]:
df=pd.read_csv('Womens Clothing E-Commerce Reviews.csv',keep_default_na=False)
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [22]:
#Basic Data PreProcessing

In [23]:
df['Review']=(df['Title'].map(str)+' '+df['Review Text']).apply(lambda row: row.strip())
df['Rating']=[1 if rating>3 else 0 for rating in df['Rating']]
df=df[['Review','Rating']]
df

,Review,Rating
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,Some major design flaws I had such high hopes ...,0
3,"My favorite buy! I love, love, love this jumps...",1
4,Flattering shirt This shirt is very flattering...,1
...,...,...
23481,Great dress for many occasions I was very happ...,1
23482,Wish it was made of cotton It reminds me of ma...,0
23483,"Cute, but see through This fit well, but the t...",0
23484,"Very cute dress, perfect for summer parties an...",0


In [28]:
df=df[df['Review']!='']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22642 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  22642 non-null  object
 1   Rating  22642 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 530.7+ KB


In [31]:
df['Rating'].value_counts()

1    17449
0     5193
Name: Rating, dtype: int64

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df[['Review']],df['Rating'],random_state=42)
X_train.shape, X_test.shape

((16981, 1), (5661, 1))

In [38]:
y_train.shape,y_test.shape

((16981,), (5661,))

In [35]:
from collections import Counter
Counter(y_train),Counter(y_test)

(Counter({1: 13059, 0: 3922}), Counter({1: 4390, 0: 1271}))

In [39]:
y_train.value_counts()

1    13059
0     3922
Name: Rating, dtype: int64

In [40]:
#Experiment 1: Basic NLP Count Based Features
#WordCount
#CharacterCount
#Word Density
#PunctuationCount
#UppercaseCount
#TitleWordCount

#Ref: https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

In [60]:
import string

X_train['char_count']=X_train['Review'].apply(len)
X_train['word_count']=X_train['Review'].apply(lambda x:len(x.split()))
X_train['word_density']=X_train['char_count']/(X_train['word_count']+1)
X_train['punctuation_count']=X_train['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
X_train['title_word_count']=X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count']=X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

X_test['char_count']=X_test['Review'].apply(len)
X_test['word_count']=X_test['Review'].apply(lambda x:len(x.split()))
X_test['word_density']=X_test['char_count']/(X_train['word_count']+1)
X_test['punctuation_count']=X_test['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
X_test['title_word_count']=X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count']=X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [61]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
12896,Soooo soft! This is a delightfully soft and fl...,268,52,5.056604,8,2,0
13183,"Had my eye on this, but dind't get I finally v...",399,84,4.694118,20,2,1
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0
13366,"Boxy. large. Boxy, unflattering, and large.\n\...",295,51,5.673077,22,2,0
